In [1]:
import pygmsh

In [2]:
resolution = 2

In [3]:
# Initialize empty geometry using the build in kernel in GMSH
geometry = pygmsh.geo.Geometry()
# Fetch model we would like to add data to
model = geometry.__enter__()

In [4]:
# Add points with finer resolution on left side
points1 = [
          model.add_point((0, 0, 0), mesh_size=resolution),
          model.add_point((10, 0, 0), mesh_size=resolution),
          model.add_point((10, 15, 0), mesh_size=resolution),
          model.add_point((0, 15, 0), mesh_size=resolution)
]

# Add lines between all points creating the rectangle
channel_lines1 = [model.add_line(points1[i], points1[i+1])
                 for i in range(-1, len(points1)-1)]

# Create a line loop and plane surface for meshing
channel_loop1 = model.add_curve_loop(channel_lines1)
plane_surface1 = model.add_plane_surface(channel_loop1)


# Add points with finer resolution on left side
points2 = [
          model.add_point((10, 0, 0), mesh_size=resolution),
          model.add_point((20, 0, 0), mesh_size=resolution),
          model.add_point((20, 15, 0), mesh_size=resolution),
          model.add_point((10, 15, 0), mesh_size=resolution)
]

# Add lines between all points creating the rectangle
channel_lines2 = [model.add_line(points2[i], points2[i+1])
                 for i in range(-1, len(points2)-1)]

# Create a line loop and plane surface for meshing
channel_loop2 = model.add_curve_loop(channel_lines2)
plane_surface2 = model.add_plane_surface(channel_loop2)

model.add_physical([plane_surface1], "cells of group 1")
model.add_physical([plane_surface2], "cells of group 2")

# Call gmsh kernel before add physical entities
model.synchronize()

In [5]:
geometry.generate_mesh(dim=2, algorithm=8)
import gmsh
gmsh.write("p16.mesh")
gmsh.fltk.run()
gmsh.clear()
geometry.__exit__()

In [6]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
#     cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
    return out_mesh

In [7]:
import meshio
mesh_from_file = meshio.read("p16.mesh")

In [8]:
triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=True)

In [9]:
meshio.write("p16.mesh", triangle_mesh)